In [17]:
from transformers import pipeline
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os


In [18]:
load_dotenv()

True

In [19]:
def img2text(url):
    img_to_text_pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
    text = img_to_text_pipe(url)[0]["generated_text"]
    # print(text)
    return text

In [20]:
def object_detection(imgUrl):
    # Use a pipeline as a high-level helper
    object_detection_pipe = pipeline("object-detection", model="facebook/detr-resnet-50")
    # Pass the image URL and the text prompt to the model
    results = object_detection_pipe(imgUrl)
    labels = [item['label'] for item in results]

    return labels

In [21]:
scenario = object_detection("static/imgs/library.jpg")
print(scenario)

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argum

ValueError: Incorrect image source. Must be a valid URL starting with `http://` or `https://`, a valid path to an image file, or a base64 encoded string. Got static/imgs/library.jpg. Failed with Incorrect padding

In [8]:
def firstObject():
    image_folder = 'static/imgs/shots'
    # List all image filenames in the folder
    images = os.listdir(image_folder)
    images.sort()
    if(images):
        scenario  = object_detection('static/imgs/shots/' + images[-1])
        unique_scenario_list = list(set(scenario))
        object_list = []
        for item in unique_scenario_list:
            if(item!="person"):
                object_list.append(item)
        if (object_list):
            return object_list[0]
        else:
            return "none"
    else:
        return "no image"

In [11]:
scenario = firstObject()
print(scenario)

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argum

apple


In [22]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=200,
    timeout=None,
    max_retries=2
)

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an advocate of sustainability. With the object given, come up with a creative way to reuse this object. ",
        ),
        ("human", "{scenario_lang}"),
    ]
)

In [24]:
chain = prompt | llm | StrOutputParser()

chain.invoke({
    "scenario_lang" : scenario
})

'An apple, while primarily a delicious and nutritious snack, can be creatively reused in several sustainable ways once it has served its primary purpose. Here’s a creative idea to extend the life of an apple:\n\n**Apple Cider Vinegar Production:**\n\nInstead of discarding apple cores and peels, you can use them to make homemade apple cider vinegar. This not only reduces waste but also provides you with a versatile product that can be used for cooking, cleaning, and even personal care. Here’s how you can do it:\n\n1. **Collect Apple Scraps:** Save the cores and peels from apples you’ve eaten or used in cooking. Make sure they are clean and free from any mold or rot.\n\n2. **Fermentation Jar:** Place the apple scraps in a large, clean glass jar. Fill the jar about halfway with the scraps.\n\n3. **Add Sugar and Water:** Dissolve a tablespoon of sugar in a cup of water and pour it over the apple scraps. Add more water'

In [34]:
def societal_definition(type):
    society_dict = {
        "Growing": "Focused on expansion, development, and continued acceleration.",
        "Collapsing": "Focused on the decline or breakdown of society, potentially leading to a simpler society.",
        "Controlling": "Focused on managing and controlling societal excesses to preserve resources and maintain order, guided by various values and authoritarian control.",
        "Transforming": "Focused on embracing radical changes that reshapes humanity and the environment, potentially leading to a posthuman future."
    }
    return society_dict.get(type,"")

In [35]:
def textGeneration_langChain(msg,type):
    """
    msg is the scenario for the story from the pic (hugging face model output);
    type is the sustainability suggestion mode - Growing, Collapsing, Controlling, Transforming
    """
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.2,
        max_tokens=200,
        timeout=None,
        max_retries=2
    )

    defintion = societal_definition(type)

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an advocate of sustainability. With the object given, come up with a way to reuse this object in the future. From the viewpoint of a society that is {suggestion_mode}. A {suggestion_mode} society is defined as one that is: {societal_definition}."
            ),
            (
                "human", 
                "{scenario_lang}"
            ),
        ]
    )

    chain = prompt | llm | StrOutputParser()

    out_message = chain.invoke({
        "suggestion_mode" : type,
        "scenario_lang" : msg,
        "societal_definition": defintion,
    })

    return out_message

    

In [36]:
story = textGeneration_langChain(scenario,"Collapsing")
print(story)

In a collapsing society, where resources are scarce and sustainability is crucial, an apple can be repurposed in several innovative ways to support a simpler, self-sufficient lifestyle:

1. **Seed Propagation**: After consuming the apple, the seeds can be collected, dried, and planted to grow new apple trees. This not only provides a renewable source of food but also contributes to reforestation efforts, which can help stabilize local ecosystems.

2. **Composting**: The apple core and any other organic waste can be added to a compost pile. Over time, this will break down into nutrient-rich soil that can be used to fertilize gardens, enhancing food production in a sustainable manner.

3. **Natural Cleaning Agent**: The acidity in apple peels can be used as a natural cleaning agent. By soaking the peels in water, you can create a mild cleaning solution that can be used for household cleaning tasks, reducing the need for chemical cleaners.

4. **Pectin Extraction


In [37]:
def runModels_langchain(url, type):
    scenario = firstObject()
    story = textGeneration_langChain(scenario,type)
    return([scenario,story])

In [38]:
(caption, story) = runModels_langchain("static/imgs/shots/library.jpg","Controlling")
print(story)

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argum

In a Controlling society focused on managing resources and maintaining order, the reuse of an apple can be approached with a structured and systematic plan to minimize waste and maximize utility. Here’s a way to reuse an apple in such a society:

1. **Comprehensive Utilization Program**: Establish a program where every part of the apple is utilized to its fullest potential, ensuring that nothing goes to waste. This program would be strictly monitored and enforced by local authorities to ensure compliance.

2. **Nutritional Allocation**: Apples would be distributed based on nutritional needs, with portions allocated to individuals according to their dietary requirements. This ensures that the fruit is consumed efficiently and supports the health of the population.

3. **Peel and Core Processing**: The peels and cores, often discarded, would be collected and processed into secondary products. For example, apple peels can be dried and ground into a powder to be used as a natural sweetener